# 3D Plotting

This tutorial is an introduction to using Neuropythy to plot the 3D cortices of either FreeSurfer or HCP subjects. 

**Author**: &nbsp;&nbsp; [Noah C. Benson](mailto:nben@uw.edu)  
**Date**: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; June 04, 2022  
**Link**: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [noahbenson/neuropythy-tutorials](https://github.com/noahbenson/neuropythy-tutorials)

## Setup

To start with, we need to import various libraries. These include, first, neuropythy itself, and, second, the ipyvolume library for 3D plotting. We also import various utility libraries like os and sys.

In [1]:
# Import some standard/utility libraries:
import os, sys, six # six provides python 2/3 compatibility

# Import our numerical/scientific libraries, scipy and numpy:
import numpy as np
import scipy as sp

# The neuropythy library is a swiss-army-knife for handling MRI data, especially
# anatomical/structural data such as that produced by FreeSurfer or the HCP.
# https://github.com/noahbenson/neuropythy
import neuropythy as ny

# We also import ipyvolume, the 3D graphics library used by neurropythy, for 3D
# surface rendering (optional).
import ipyvolume as ipv

## Choose a Subject

We need a subject whose ROIs we are going to draw. This notebook uses the subject 'S1202' from from [Benson and Winawer (2018)](https://doi.org/10.7554/eLife.40224), which neuropythy will automatically download for you. The first time you download this, it will likely take some time as neuropythy needs to download and unzip most of the dataset. This will be cached if possible: If you are using the Docker container to run this tutorial, the cache will only be cleared whenever the docker-image is rebuilt; otherwise, if you don't have a path configured for the neuropythy config item `'data_cache_root'`, these data will be downloaded fresh each time you run the notebook (and will be auto-deleted when your python kernel exits). To check this, you can look at `ny.config['data_cache_root']`. This can be set to a valid directory to store various neuropythy auto-downloaded data. The file `~/.npythyrc` containing a dictionary in JSON format may be given the key `"data_cache_root"` to ensure that this configuration is preserved across sessions. The data downloaded by neuropythy to this directory may be deleted without consequence (aside from needing to re-download it again later).

In [2]:
# Note: This may take awhile to download if you have not downloaded these
# data previously (~9 GB once extracted).
sub = ny.data['benson_winawer_2018'].subjects['S1202']

Alternately, if you want to use a subject of your own, you can provide neuropythy with the subject's path, as in the code-block below.

```python
sub_path = '/data/cache/benson_winawer_2018/freesurfer_subjects/S1202'
sub = ny.freesurfer_subject(sub_path)
```

## Plotting a Single Hemisphere

### Basic Plotting

If we want to see a property visualized on the cortical surface, we can plot the cortical surface using that property as the color.

In [5]:
# We'll plot the thickness property, with a minimum value
# (for the colormap) of 2 mm and a max of 6 mm.
ny.cortex_plot(sub.lh, surface='inflated', color='thickness',
               cmap='hot', vmin=2, vmax=6)

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

Notice that in the above code cell, we passed the string `'thickness'` to the `color` option. This can be a property name for a property of the first argument (the cortex or surface mesh), or any valid property itself. For example, we could pass the thickness property directly.

In [6]:
thickness = sub.lh.prop('thickness')
ny.cortex_plot(sub.lh, surface='inflated', color=thickness,
               cmap='hot', vmin=2, vmax=6)

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

Similarly, the first argument of the `cortex_plot` function is flexible. It may be either a `Cortex` object such as `sub.lh` or it can be a `Mesh` object such as `sub.lh.white_surface`. If it is a `Cortex` object, then the `surface` option should be given the name of a surface of the `Cortex`.

In [34]:
# Here, we pass the inflated surface directly to the cortex_plot
# function instead of providing the surface option.
ny.cortex_plot(sub.lh.surface('inflated'), color='thickness',
               cmap='hot', vmin=2, vmax=6)

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

### Masks and Underlays

The `cortex_plot` function accepts an option `mask` that allows one to color the cortex only at certain places. A very basic usage of the mask is to plot only the vertices that are `True` in a particular property.

In [35]:
# Plot only on the cortical surface (i.e., exclude the Corpus callosum).
# The 'cortex_label' property stores True values for cortex and False
# values for the medial wall.
ny.cortex_plot(sub.lh, surface='midgray', color='thickness',
               cmap='hot', vmin=2, vmax=6,
               mask=sub.lh.prop('cortex_label'))

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

The mask can be a property vector itself, but alternately, you can provide the name of a property that is already part of the cortex or surface object.

In [37]:
ny.cortex_plot(sub.lh, surface='midgray', color='thickness',
               cmap='hot', vmin=2, vmax=6,
               mask='cortex_label')

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

Mask also accepts a few alternative forms; these are described in the `ny.to_mask` function's documentation.

In [18]:
help(ny.to_mask)

Help on function to_mask in module neuropythy.geometry.mesh:

to_mask(obj, m=None, indices=None)
    to_mask(obj, m) yields the set of indices from the given vertex-set or itable object obj that
      correspond to the given mask m.
    to_mask((obj, m)) is equivalent to to_mask(obj, m).
    
    The mask m may take any of the following forms:
       * a list of vertex indices
       * a boolean array (one value per vertex)
       * a property name, which can be cast to a boolean array
       * a tuple (property, value) where property is a list of values, one per vertex, and value
         is the value that must match in order for a vertex to be included (this is basically
         equivalent to the mask (property == value); note that property may also be a property
         name
       * a tuple (property, min, max), which specifies that the property must be between min and
         max for a vertex to be included (min < p <= max)
       * a tuple (property, (val1, val2...)), which sp

Here's an example where we only plot the thickness values that are greater than 0 by using the `(property, min, max)` form for the mask.

In [36]:
ny.cortex_plot(sub.lh, surface='inflated', color='thickness',
               cmap='hot', vmin=2, vmax=6,
               mask=('thickness', 0, np.inf))

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

Suppose you want to color only certain points on cortex, but you don't want the underly to be colored by curvature. The solution to this problem is to use the `underlay` option, which allows one to specify a color for the underlay.

In [25]:
ny.cortex_plot(sub.lh, surface='inflated', color='thickness',
               cmap='hot', vmin=2, vmax=6,
               mask=('thickness', 0, np.inf),
               underlay='white')

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

## Plotting Multiple Hemispheres

To plot multiple hemispheres simultaneously, we can use a number of techniques. The first, is to treat the `ipyvolume` figure a bit like a `matplotlib` axes object. Essentially, we can pass the figure object to the `cortex_plot` function in order to plot two hemispheres together.

In [26]:
fig = ipv.figure()

ny.cortex_plot(sub.lh, surface='pial', figure=fig)
ny.cortex_plot(sub.rh, surface='pial', figure=fig)

ipv.show()

Alternately, we can just pass both hemispheres to the `cortex_plot` function. This will work for simple plots, but for more complex plots--in which the cortical surfaces aren't colored the same way, for example--you'll typically want to use the `figure` method (above) when you are plotting different things on each hemisphere. 

In [10]:
ny.cortex_plot((sub.lh, sub.rh), surface='midgray')

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

### Transparency

Although it is only sometimes useful, neuropythy supports transparency for both the mesh and the color. The `alpha` parameter allows one to modify the transparency of the vertex colors.

In [30]:
ny.cortex_plot(sub.lh, surface='midgray', color='thickness',
               cmap='hot', vmin=2, vmax=6,
               mask=sub.lh.prop('cortex_label'),
               alpha=1 - sub.lh.prop('V2_weight'))

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

Alternately, we can make the mesh transparent. This might be useful if we want to show both the white and the pial matter, for example.

In [33]:
fig = ipv.figure()

ny.cortex_plot(sub.lh.white_surface, figure=fig)
ny.cortex_plot(sub.lh.pial_surface, figure=fig,
               mesh_alpha=0.5, underlay='gray')

ipv.show()

## More Information

More help on the plotting functions can be found in the documentation for the `cortex_plot` function:

In [8]:
help(ny.cortex_plot)

Help on function cortex_plot in module neuropythy.graphics.core:

cortex_plot(mesh, *args, **opts)
    cortex_plot(mesh) calls either cortex_plot_2D or cortex_plot_3D depending on the dimensionality
      of the given mesh, and yields the resulting graphics object. All optional arguments supported
      by each is supported by cortex plot.
    
    The following options are accepted:
      * color (default: None) specifies the color to plot for each vertex; this argument may take a
        number of forms:
          * None, do not plot a color over the underlay (the default)
          * a matrix of RGB or RGBA values, one per vertex
          * a property vector or a string naming a property, in which case the cmap, vmin, and vmax
            arguments are used to generate colors
          * a function that, when passed a single argument, a dict of the properties of a single
            vertex, yields an RGB or RGBA list for that vertex.
      * cmap (default: 'eccenflat') specifies th